# TODO's
* run it with more frames

In [1]:
import pandas
from tabulate import tabulate
from IPython.core.display import display, HTML
from statistics import mean
from collections import Counter

In [4]:
import look_up_utils
import createq_utils

In [11]:
from glob import glob
import os
all_frames = [os.path.basename(path)
              for path in glob('../EventRegistries/GunViolence/frames/*')
              if not 'Icon' in path
             ]
all_frames

['accidental_deaths',
 'accidental_deaths_children',
 'accidental_deaths_teens',
 'accidental_injuries',
 'accidental_injuries_children',
 'accidental_injuries_teens',
 'children_injured',
 'children_killed',
 'mass_shootings',
 'mass_shootings_2013',
 'mass_shootings_2014',
 'mass_shootings_2015',
 'officer_involved_shootings',
 'teens_injured',
 'teens_killed']

## Load frames

In [12]:
frames = all_frames
df = pandas.concat([pandas.read_pickle('../EventRegistries/GunViolence/frames/' + frame)
                    for frame in frames])

## Load look_up

In [13]:
look_up, parameters2incident_uris = look_up_utils.create_look_up(frames)

## Create table

In [14]:
lists_of_lists = []
wanted_attrs = ['answer', 'n2s_ratio', 
                'a_avg_date_spread', 'a_avg_num_sources', 
                'n_avg_date_spread', 'n_avg_num_sources']

headers = ['confusion', 'min_confusion', '#cand', 'granularity'] + wanted_attrs 

for confusion_tuple in [('location', 'time'),
                        ('location', 'participant'),
                        ('participant', 'time'),
                        ]:
    for min_confusion in range(2, 4):
        print(confusion_tuple, min_confusion)
        candidates=createq_utils.lookup_and_merge(look_up, 
                                                  parameters2incident_uris,
                                                  confusion_tuple,
                                                  min_confusion,
                                                  df,
                                                  debug=False,
                                                  inspect_one=False) 
        
        gran_distr = Counter([getattr(cand, 'granularity') 
                              for cand in candidates])
        
        one_row = [confusion_tuple, min_confusion, len(candidates), gran_distr]
        
        for attr in wanted_attrs:
            values = [getattr(cand, attr) for cand in candidates]
            row_value = (round(min(values), 1), 
                         round(mean(values), 1), 
                         round(max(values), 1))
            one_row.append(row_value)
        
        lists_of_lists.append(one_row)

stats_df = pandas.DataFrame(lists_of_lists, columns=headers)
table = tabulate(stats_df, headers='keys', tablefmt='html')
display(HTML(table))

('location', 'time') 2
('location', 'time') 3
('location', 'participant') 2
('location', 'participant') 3
('participant', 'time') 2
('participant', 'time') 3


,confusion,min_confusion,#cand,granularity,answer,n2s_ratio,a_avg_date_spread,a_avg_num_sources,n_avg_date_spread,n_avg_num_sources
0,"('location', 'time')",2,202,"Counter({('city', 'year'): 114, ('city', 'month'): 59, ('address', 'year'): 11, ('city', 'day'): 8, ('address', 'day'): 5, ('address', 'month'): 5})","(2, 3.6, 29)","(0.5, 400.0, 1224.5)","(0, 24.2, 1053.5)","(0.5, 2.0, 10.0)","(0, 30.5, 232.3)","(1.6, 2.1, 3.0)"
1,"('location', 'time')",3,24,"Counter({('city', 'year'): 21, ('city', 'month'): 3})","(3, 7.1, 25)","(81.5, 293.9, 611.0)","(0, 28.3, 328.1)","(1.5, 2.1, 3.2)","(2.9, 28.4, 46.3)","(1.7, 2.1, 2.3)"
2,"('location', 'participant')",2,1250,"Counter({('state', 'first'): 524, ('state', 'last'): 453, ('city', 'last'): 112, ('city', 'first'): 95, ('state', 'full_name'): 34, ('city', 'full_name'): 24, ('address', 'first'): 3, ('address', 'last'): 3, ('address', 'full_name'): 2})","(2, 2.5, 13)","(0.0, 83.2, 203.5)","(0, 93.8, 15998.5)","(0.5, 3.3, 17.5)","(-1, 45.0, 358.3)","(0, 2.3, 7.8)"
3,"('location', 'participant')",3,336,"Counter({('state', 'first'): 157, ('state', 'last'): 146, ('city', 'last'): 20, ('city', 'first'): 9, ('state', 'full_name'): 3, ('city', 'full_name'): 1})","(3, 3.8, 13)","(0.3, 70.2, 139.7)","(0, 88.5, 6400.6)","(1.0, 3.5, 10.3)","(0, 45.3, 166.3)","(1.6, 2.3, 4.8)"
4,"('participant', 'time')",2,2413,"Counter({('first', 'year'): 818, ('last', 'year'): 644, ('first', 'month'): 431, ('last', 'month'): 366, ('last', 'day'): 41, ('first', 'day'): 40, ('full_name', 'year'): 36, ('full_name', 'month'): 29, ('full_name', 'day'): 8})","(2, 3.2, 41)","(0.5, 390.7, 1226.0)","(0, 80.6, 16003)","(0.5, 3.4, 14.5)","(0, 44.9, 322.8)","(1.0, 2.3, 7.0)"
5,"('participant', 'time')",3,941,"Counter({('first', 'year'): 417, ('last', 'year'): 305, ('first', 'month'): 118, ('last', 'month'): 95, ('full_name', 'year'): 2, ('full_name', 'month'): 2, ('last', 'day'): 1, ('first', 'day'): 1})","(3, 5.0, 41)","(9.0, 318.5, 818.7)","(0, 71.6, 6417.2)","(1.0, 3.5, 13.7)","(0.9, 44.5, 322.8)","(1.7, 2.2, 3.8)"


In [16]:
pandas.to_pickle(stats_df, 'stats.pickle')